In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import requests
import json
import time
import random
import threading
import pandas as pd
import datetime # 날짜/시간 관리를 위한 모듈

# ================================================================
# 🚨🚨 중요: 아래 _BUILD_ID_ 및 헤더 값은 사용자 환경에 따라 변경될 수 있습니다.
# 프로그램 실행 전, 아래 주석을 참고하여 값을 확인/갱신해 주세요. 🚨🚨
# ================================================================

# 네이버 부동산 웹사이트(fin.land.naver.com) 개발자 도구(F12) -> Network 탭 ->
# 'regions.json' 요청 찾기 -> 'Headers' 탭의 Request URL에서 "_next/data/" 뒤의 값 복사
_BUILD_ID_ = "KVZ8_AwgDYT1YkrfeHfcs" # <<-- !!! 여기에 최신 _BUILD_ID_를 붙여넣으세요 !!!
# _BUILD_ID_는 웹사이트 업데이트 시 변경될 수 있습니다. 

# 프로그램 사용 만료일 설정 (년, 월, 일)
# 이 날짜가 지나면 프로그램 사용이 제한됩니다.
EXPIRATION_DATE = datetime.date(2025, 12, 31) # <--!!지정일까지 사용 가능!!

# API 엔드포인트
REGION_API_BASE_URL = f"https://fin.land.naver.com/_next/data/{_BUILD_ID_}/regions.json" 
GET_API_URL = "https://m.land.naver.com/cluster/ajax/articleList" 
GET_REGION_LIST_API_URL = "https://m.land.naver.com/map/getRegionList" 

# HTTP 요청 헤더 설정
# 이 헤더 값들은 브라우저에서 웹페이지에 접속할 때 자동으로 전송되는 정보입니다.
# 웹사이트의 보안 정책에 따라, 이 값들이 실제 브라우저와 다르면 요청이 거부될 수 있습니다.
# 특히 'Cookie'는 사용자의 로그인 세션 정보를 담고 있으므로 매우 민감한 정보입니다.
# -------------------------------------------------------------------------------------

# m.land.naver.com 도메인 API 호출 시 사용 (매물 조회, getRegionList 용)
DEFAULT_HEADERS = {
    # Accept: 서버로부터 어떤 형태의 응답을 받을지 정의합니다. 일반적으로 '*'/* 로 모든 타입을 허용합니다. (구동에 필수)
    #"Accept": "application/json, text/plain, */*",
    # Accept-Encoding: 어떤 압축 방식으로 응답을 받을지 정의합니다. gzip, deflate가 가장 일반적입니다. (구동에 필수)
    #"Accept-Encoding": "gzip, deflate, br, zstd",
    # Accept-Language: 선호하는 언어입니다. 한글 웹페이지 요청 시 주로 사용됩니다. (구동에 영향 미칠 수 있음)
    #"Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    # Connection: 클라이언트와 서버 간의 연결 방식을 제어합니다. keep-alive는 연결 유지를 요청합니다. (구동에 필수)
    #"Connection": "keep-alive",
    # Host: 요청을 보내는 서버의 도메인 이름입니다. (구동에 필수)
    "Host": "m.land.naver.com",
    # Origin: 요청이 시작된 도메인을 나타냅니다. 웹 보안(CORS) 관련하여 중요한 역할을 합니다. (구동에 영향 미칠 수 있음)
    # Origin 값은 Referer와 함께 봇 감지에 사용될 수 있습니다.
    #"Origin": "https://m.land.naver.com", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    # Referer: 현재 요청이 어느 페이지에서 시작되었는지 나타냅니다. (구동에 영향 미칠 수 있음)
    # 웹사이트가 스크래핑을 방지하는 데 사용될 수 있으며, 사용자 활동 추적에 사용될 수 있습니다.
    #"Referer": "https://m.land.naver.com/", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    # User-Agent: 요청을 보내는 사용자(클라이언트 소프트웨어)의 정보를 나타냅니다. (구동에 영향 미칠 수 있음)
    # 특정 OS, 브라우저 버전으로 위장하여 봇 감지를 회피하는 데 사용되기도 하며, 사용자 식별에 활용될 수 있습니다.
    "User-Agent": "A", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    # Sec-Fetch-Dest, Mode, Site: Fetch Metadata Request Headers로, 보안 및 요청 출처를 강화합니다.
    # 브라우저가 자동으로 추가하는 경우가 많습니다. (구동에 영향 미칠 수 있음)
    #"Sec-Fetch-Dest": "empty", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    #"Sec-Fetch-Mode": "cors",   # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    #"Sec-Fetch-Site": "same-origin", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
    # Cookie: !!! 가장 민감한 개인 정보입니다 !!! (구동에 치명적인 영향 미칠 수 있음)
    # 사용자의 로그인 세션 정보 등을 담고 있어 웹사이트가 사용자를 식별하는 데 사용합니다.
    # 프로그램 실행 전에 반드시 최신 유효한 값으로 교체해야 하며, 주기적으로 만료될 수 있습니다.
    # 이 라인을 주석 처리할 경우 프로그램 구동이 안될 확률이 매우 높습니다. 테스트 후 필요 시 해제하세요.
    #"Cookie": "NAC=LcV5BogfnITfC; NNB=53JOHN6SM5RGQ; SHOW_FIN_BADGE=Y; bnb_tooltip_shown_finance_v1=true; _fwb=220FhMGmpQ2BtTNICGStK6Z.1752616946035; landHomeFlashUseYn=Y; ASID=7a2b3579000001985a70dc080000001b; nhn.realestate.article.rlet_type_cd=A01; nhn.realestate.article.trade_type_cd=\"\"; realestate.beta.lastclick.cortar=4400000000; PROP_TEST_KEY=1754632854081.db0b658403da2bf0f24b506613cf8d9acf878c7af22296020db8166f2844b2db; PROP_TEST_ID=44995f4422776c30b611ccf580060a39260c013b9d98179a5129d683c639425e5; NACT=1; SRT30=1755429025; SRT5=1755429025; BUC=e7PyShvnIwFwEw29RktTtkcu22vOGmmbFptWmcnbqyM=", # 🔒 개인 정보/추적성 관련 (구동 안될 시 해제 요망)
}


class NaverRealEstateApp:
    # 부동산 유형 코드 매핑 (한글 이름 -> API 코드)
    _RLET_TYPE_CODES = {
        '아파트': 'APT',
        '빌라': 'VL:YR:DSD',  # 빌라 선택 시 빌라, 연립, 다세대 모두 포함
        '오피스텔': 'OPST',
    }

    # 거래 유형 코드 매핑 (한글 이름 -> API 코드)
    _TRAD_TYPE_CODES = {
        '매매': 'A1',
        '전세': 'B1',
        '월세': 'B2',
    }

    # 평형대 코드 및 spcMin/spcMax 매핑 (사용자 표시명 -> {spcMin, spcMax})
    _PYEONG_TYPE_RANGES = {
        '~10평': {'spcMax': 33},
        '10평~20평': {'spcMin': 33, 'spcMax': 66},
        '20평~30평': {'spcMin': 66, 'spcMax': 99},
        '30평~40평': {'spcMin': 99, 'spcMax': 132},
        '40평~50평': {'spcMin': 132, 'spcMax': 165},
        '50평~60평': {'spcMin': 165, 'spcMax': 198},
        '60평~70평': {'spcMin': 198, 'spcMax': 231},
        '70평~': {'spcMin': 231, 'spcMax': 900000000}, # 매우 큰 값으로 무제한 의미
    }

    # 줌 레벨에 따른 대략적인 델타 값 (시행착오를 통해 조정 필요)
    _Z_LEVEL_DELTAS = {
        10: {'lat_delta': 0.2, 'lon_delta': 0.2},
        11: {'lat_delta': 0.1, 'lon_delta': 0.1},
        12: {'lat_delta': 0.05, 'lon_delta': 0.05}, 
        13: {'lat_delta': 0.025, 'lon_delta': 0.025},
        14: {'lat_delta': 0.012, 'lon_delta': 0.012},
        15: {'lat_delta': 0.006, 'lon_delta': 0.006},
        16: {'lat_delta': 0.003, 'lon_delta': 0.003},
    }

    def __init__(self, master):
        self.master = master
        master.title("네이버 부동산 매물 조회 앱")
        master.geometry("1200x780") 
        master.resizable(True, True)

        self._stop_flag = False
        self._regions_data = {} 
        self.fetched_article_data = [] 

        # 입력 프레임 구성
        input_frame = tk.Frame(master, padx=10, pady=10, relief="groove", bd=2)
        input_frame.pack(pady=10, fill="x")

        # ----- 제1세로열: 지역 선택 콤보박스 (Column 0-1) -----
        # 시작 행: 0
        combobox_label_col = 0 
        combobox_widget_col = 1 

        current_row_for_combobox = 0 
        tk.Label(input_frame, text="시/도 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.sido_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.sido_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="시/군/구 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.gungu_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.gungu_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.gungu_combobox.bind("<<ComboboxSelected>>", self.on_gungu_selected)

        current_row_for_combobox += 1
        tk.Label(input_frame, text="법정동 선택:", font=('맑은 고딕', 10, 'bold')).grid(row=current_row_for_combobox, column=combobox_label_col, padx=5, pady=5, sticky="w")
        self.legal_dong_combobox = ttk.Combobox(input_frame, state="readonly", font=('맑은 고딕', 10))
        self.legal_dong_combobox.grid(row=current_row_for_combobox, column=combobox_widget_col, padx=5, pady=5, sticky="ew")
        self.legal_dong_combobox.bind("<<ComboboxSelected>>", self.on_legal_dong_selected)

        # 법정동 코드 저장을 위한 StringVar (UI에 직접 노출X)
        self.cortar_no_var = tk.StringVar() 
        # 좌표 정보 저장용 (UI에 직접 노출X)
        self.coord_vars = {
            'lat': tk.StringVar(), 'lon': tk.StringVar(), 'btm': tk.StringVar(),
            'lft': tk.StringVar(), 'top': tk.StringVar(), 'rgt': tk.StringVar()
        }

        # ----- 중앙 영역: 체크박스 그룹들 (Column 2-7) -----
        checkbox_label_start_col = 2 
        checkbox_widget_start_col = 3 

        checkbox_current_row = 0 

        # --- 부동산 유형 (체크박스 그룹) ---
        tk.Label(input_frame, text="부동산 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        rlet_type_checkbox_frame = tk.Frame(input_frame)
        rlet_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.rlet_type_vars = {} 
        for name in self._RLET_TYPE_CODES.keys():
            var = tk.BooleanVar(value=(name == '아파트')) 
            cb = tk.Checkbutton(rlet_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.rlet_type_vars[name] = var

        # --- 거래 유형 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="거래 유형:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        trad_type_checkbox_frame = tk.Frame(input_frame)
        trad_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w")
        self.trad_type_vars = {} 
        for name in self._TRAD_TYPE_CODES.keys():
            var = tk.BooleanVar(value=True) 
            cb = tk.Checkbutton(trad_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.pack(anchor="w", side="left") 
            self.trad_type_vars[name] = var
        
        # --- 평형대 (체크박스 그룹) ---
        checkbox_current_row += 1
        tk.Label(input_frame, text="평형대:", font=('맑은 고딕', 10, 'bold')).grid(row=checkbox_current_row, column=checkbox_label_start_col, padx=10, pady=5, sticky="w")
        pyeong_type_checkbox_frame = tk.Frame(input_frame)
        pyeong_type_checkbox_frame.grid(row=checkbox_current_row, column=checkbox_widget_start_col, padx=5, pady=5, sticky="w", rowspan=2) # 평형대는 2줄에 걸쳐서 배치
        self.pyeong_type_vars = {} 
        
        # 평형대 체크박스를 2행으로 나누어 나열
        pyeong_names = list(self._PYEONG_TYPE_RANGES.keys())
        num_cols_per_row_in_frame = (len(pyeong_names) + 1) // 2 
        
        for i, name in enumerate(pyeong_names):
            row_in_frame = i // num_cols_per_row_in_frame 
            col_in_frame = i % num_cols_per_row_in_frame 
            
            var = tk.BooleanVar(value=False) 
            cb = tk.Checkbutton(pyeong_type_checkbox_frame, text=name, variable=var, font=('맑은 고딕', 10))
            cb.grid(row=row_in_frame, column=col_in_frame, padx=5, sticky="w") 
            self.pyeong_type_vars[name] = var

        # ----- 오른쪽 영역: 설정 및 버튼들 (Column 8-9) -----
        right_section_label_col = 8 
        right_section_widget_col = 9 

        # 줌 레벨 설정 (UI에서 숨기기)
        row_offset = 0 
        self.z_level_var = tk.IntVar(value=12) # 기본값 12로 설정
        # tk.Label(input_frame, text="줌 레벨 (Z):", font=('맑은 고딕', 10, 'bold')).grid(row=row_offset, column=right_section_label_col, padx=10, pady=5, sticky="w")
        # tk.Entry(input_frame, textvariable=self.z_level_var, width=10, font=('맑은 고딕', 10)).grid(row=row_offset, column=right_section_widget_col, padx=5, pady=5, sticky="ew")

        # 최대 페이지 설정 (UI에서 숨기기)
        row_offset += 1
        self.max_pages_var = tk.IntVar(value=50) # 기본값 50으로 설정
        # tk.Label(input_frame, text="최대 페이지:", font=('맑은 고딕', 10, 'bold')).grid(row=row_offset, column=right_section_label_col, padx=10, pady=5, sticky="w")
        # tk.Entry(input_frame, textvariable=self.max_pages_var, width=10, font=('맑은 고딕', 10)).grid(row=row_offset, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        
        # --- 버튼 재배치 ---
        # 매물 조회 버튼 (콤보박스 아래에 배치)
        query_button_row = current_row_for_combobox + 1 
        self.query_button = tk.Button(input_frame, text="매물 조회", command=self.start_fetch_thread, bg="#007BFF", fg="white", font=('맑은 고딕', 10, 'bold'), width=10) 
        self.query_button.grid(row=query_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 

        # 중지 버튼 (매물 조회 버튼 바로 아래)
        stop_button_row = query_button_row + 1 
        self.stop_button = tk.Button(input_frame, text="중지", command=self.stop_fetch, bg="#FF4500", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED, width=10) 
        self.stop_button.grid(row=stop_button_row, column=combobox_label_col, columnspan=combobox_widget_col + 1, padx=5, pady=10, sticky="ew") 


        # 저장 공간 입력창 및 버튼 (원래의 오른쪽 섹션의 남은 공간 활용, 하지만 row는 별도로 계산하여 최상단에서 시작)
        save_section_start_row = 0 # 줌 레벨/최대 페이지가 숨겨졌으므로 이 부분이 오른쪽의 가장 위에서 시작
        tk.Label(input_frame, text="저장 경로:", font=('맑은 고딕', 10, 'bold')).grid(row=save_section_start_row, column=right_section_label_col, padx=10, pady=5, sticky="w")
        self.save_path_var = tk.StringVar(value="results.csv") 
        tk.Entry(input_frame, textvariable=self.save_path_var, width=20, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_widget_col, padx=5, pady=5, sticky="ew")
        
        save_section_start_row += 1
        tk.Button(input_frame, text="저장 경로 선택", command=self.select_save_path, font=('맑은 고딕', 10)).grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=5, sticky="ew")

        save_section_start_row += 1
        self.save_button = tk.Button(input_frame, text="조회 결과 저장", command=self.save_articles_to_file, bg="#28A745", fg="white", font=('맑은 고딕', 10, 'bold'), state=tk.DISABLED)
        self.save_button.grid(row=save_section_start_row, column=right_section_label_col, columnspan=2, padx=5, pady=10, sticky="ew")


        # 그리드 컬럼 설정 (비율)
        input_frame.grid_columnconfigure(combobox_widget_col, weight=1) 
        input_frame.grid_columnconfigure(checkbox_label_start_col, weight=0) 
        input_frame.grid_columnconfigure(checkbox_widget_start_col, weight=1) 
        input_frame.grid_columnconfigure(checkbox_label_start_col + 2, weight=0) 
        input_frame.grid_columnconfigure(checkbox_widget_start_col + 3, weight=1) 
        input_frame.grid_columnconfigure(checkbox_label_start_col + 4, weight=0) 
        input_frame.grid_columnconfigure(checkbox_widget_start_col + 5, weight=1) 
        input_frame.grid_columnconfigure(right_section_label_col, weight=0) 
        input_frame.grid_columnconfigure(right_section_widget_col, weight=1) 


        # 결과용 테이블 (이전과 동일)
        table_frame = tk.Frame(master, padx=10, pady=10)
        table_frame.pack(fill="both", expand=True)

        columns = ("atclNo", "atclNm", "tradTpNm", "hanPrc", "prc", "rentPrc", "flrInfo", "spc1", "spc2", "direction", "cortarNo")
        self.tree = ttk.Treeview(table_frame, columns=columns, show="headings")

        headers = ["매물번호", "매물명", "거래종류", "한글가격", "매매/전세가", "월세가",
                   "층정보", "공급(㎡)", "전용(㎡)", "방향", "법정동코드"]

        for col, text in zip(columns, headers):
            self.tree.heading(col, text=text)
            self.tree.column(col, width=100, anchor="center")

        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=self.tree.yview)
        vsb.pack(side="right", fill="y")
        self.tree.configure(yscrollcommand=vsb.set)

        hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=self.tree.xview)
        hsb.pack(side="bottom", fill="x")
        self.tree.configure(xscrollcommand=hsb.set)

        self.tree.pack(fill="both", expand=True)

        self.status_label = tk.Label(master, text="", bd=1, relief="sunken", anchor="w")
        self.status_label.pack(side="bottom", fill="x")

        # 지역 데이터 저장용 (sido, gungu, legal_dong)
        self._regions_data = {
            'sido': {},
            'gungu': {},
            'legal_dong': {}
        }

        # 앱 시작 후 시도 목록 자동 로드
        self.master.after(100, self.load_sido_list) 

    # 상태 메시지 업데이트 (이전과 동일)
    def update_status(self, message):
        self.status_label.config(text=message)
        self.master.update_idletasks()

    # 안전한 API 호출 함수 (이전과 동일)
    def _safe_request(self, method, url, headers, params=None, json_data=None, timeout=10, api_name="API"):
        try:
            print(f"\n--- API 요청 시작: {api_name} ---")
            print(f"  URL: {url}")
            print(f"  Method: {method.upper()}")
            print(f"  Params: {params}")
            
            if method.lower() == 'post':
                response = requests.post(url, headers=headers, json=json_data, timeout=timeout)
            elif method.lower() == 'get':
                response = requests.get(url, headers=headers, params=params, timeout=timeout)
            else:
                raise ValueError("지원하지 않는 HTTP 메서드입니다.")

            response.raise_for_status() 

            print(f"  Status Code: {response.status_code}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                delay_time = random.uniform(5, 15)
                self.update_status(f"⚠️ {api_name} 429 오류 발생. {delay_time:.2f}초 후 재시도.")
                time.sleep(delay_time)
                return "RETRY"
            else:
                response_text = e.response.text if hasattr(e.response, 'text') else '응답 본문 없음'
                self.master.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("HTTP 오류", f"{api_name} 요청 중 오류 발생:\n상태코드: {e_val.response.status_code}\nURL: {e_val.request.url}\n응답: {res_text_val}"))
                self.update_status(f"⛔️ {api_name} HTTP 오류: {e.response.status_code}")
        except requests.exceptions.ConnectionError as e:
            self.master.after(0, lambda e_val=e: messagebox.showerror("연결 오류", f"{api_name} 서버 연결 실패:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 연결 오류: {e}")
        except requests.exceptions.Timeout as e:
            self.master.after(0, lambda e_val=e: messagebox.showerror("시간 초과", f"{api_name} 요청 시간 초과:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 시간 초과: {e}")
        except requests.exceptions.RequestException as e:
            self.master.after(0, lambda e_val=e: messagebox.showerror("요청 오류", f"{api_name} 요청 중 알 수 없는 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 알 수 없는 오류: {e}")
        except json.JSONDecodeError as e:
            response_text = response.text if 'response' in locals() else '없음'
            self.master.after(0, lambda e_val=e, res_text_val=response_text: messagebox.showerror("JSON 파싱 오류", f"{api_name} 응답 JSON 디코딩 실패:\n{e_val}\n응답 본문: {res_text_val}"))
            self.update_status(f"⛔️ {api_name} JSON 파싱 오류: {e}")
        except Exception as e:
            self.master.after(0, lambda e_val=e: messagebox.showerror("알 수 없는 오류", f"{api_name} 예상치 못한 오류:\n{e_val}"))
            self.update_status(f"⛔️ {api_name} 예상치 못한 오류: {e}")
        return None

    # 모든 지역 목록을 GET_REGION_LIST_API_URL로 가져옴
    def load_sido_list(self): 
        """네이버 부동산 시도 목록 로드 (getRegionList API 사용)"""
        self.update_status("시/도 목록 로드 중...")
        
        url = GET_REGION_LIST_API_URL 
        params = {"cortarNo": "0000000000", "mycortarNo": ""} # 최상위 지역(전국) 요청
        
        data = self._safe_request('get', url, DEFAULT_HEADERS, params=params, api_name="지역 목록 API - 시도") 
        
        if data and 'result' in data and 'list' in data['result']:
            sido_list_raw = data['result']['list']
            
            sido_names = []
            self._regions_data['sido'] = {} 
            for item in sido_list_raw:
                sido_name = item.get('CortarNm')
                sido_code = item.get('CortarNo')
                if sido_name and sido_code:
                    sido_names.append(sido_name)
                    self._regions_data['sido'][sido_name] = sido_code
            
            self.sido_combobox['values'] = sido_names
            
            self.sido_combobox.unbind("<<ComboboxSelected>>")
            if sido_names:
                self.sido_combobox.set(sido_names[0]) 
                self.on_sido_selected(None) # 첫 시도 선택 후 자동으로 시군구 로드
            else:
                self.sido_combobox.set("시도 선택")
            self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected) # 다시 바인딩
            self.update_status("시/도 목록 로드 완료.")
        else:
            self.update_status("⛔️ 시/도 목록 로드 실패. (API 응답 구조 확인 필요)")
            self.master.after(0, lambda: messagebox.showerror("오류", "시도 목록을 가져오지 못했습니다. 응답 구조를 확인하세요.")) 
            self.sido_combobox['values'] = ["목록 로드 실패"]
            self.sido_combobox.set("목록 로드 실패")
        self.master.update_idletasks()

    # 시도 선택 시 시군구 목록 로드 (GET_REGION_LIST_API_URL 사용)
    def on_sido_selected(self, event):
        selected_sido_name = self.sido_combobox.get()
        selected_sido_code = self._regions_data['sido'].get(selected_sido_name)
        
        if not selected_sido_code:
            return

        self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 중...")
        # 시군구 콤보박스 및 법정동 콤보박스 초기화
        self.gungu_combobox.set('')
        self.gungu_combobox['values'] = []
        self.legal_dong_combobox.set('')
        self.legal_dong_combobox['values'] = []
        self._regions_data['gungu'] = {} 
        self._regions_data['legal_dong'] = {} 

        # 시도 코드를 cortarNo 파라미터로 사용하여 시군구 데이터 요청
        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_sido_code, "mycortarNo": ""} # 상위 지역 코드로 시도 코드 전달
        
        response_data = self._safe_request('get', url, DEFAULT_HEADERS, params=params, api_name="지역 목록 API - 시군구")
        
        if response_data and 'result' in response_data and 'list' in response_data['result']:
            gungu_list_raw = response_data['result']['list']
            gungu_names = []
            for gungu_data in gungu_list_raw:
                name = gungu_data.get('CortarNm')
                code = gungu_data.get('CortarNo')
                if name and code:
                    gungu_names.append(name)
                    # 시군구 정보와 그 하위 법정동 리스트는 나중에 다시 요청
                    self._regions_data['gungu'][name] = {'code': code}
            self.gungu_combobox['values'] = gungu_names
            self.update_status(f"'{selected_sido_name}'의 시군구 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_sido_name}'의 시군구 목록 로드 실패. (응답 구조 불일치 또는 데이터 없음)")
            self.master.after(0, lambda: messagebox.showerror("오류", f"'{selected_sido_name}' 시군구 목록을 가져오지 못했습니다. API 응답 구조를 확인해주세요."))

    # 시군구 선택 시 법정동 목록 로드 (GET_REGION_LIST_API_URL 사용)
    def on_gungu_selected(self, event):
        selected_gungu_name = self.gungu_combobox.get()
        gungu_info = self._regions_data['gungu'].get(selected_gungu_name)
        
        if not gungu_info:
            return

        selected_gungu_code = gungu_info.get('code')

        self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 중...")
        # 법정동 콤보박스 초기화
        self.legal_dong_combobox.set('')
        self.legal_dong_combobox['values'] = []
        self._regions_data['legal_dong'] = {}

        # 시군구 코드를 cortarNo 파라미터로 사용하여 법정동 데이터 요청
        url = GET_REGION_LIST_API_URL
        params = {"cortarNo": selected_gungu_code, "mycortarNo": ""}

        response_data = self._safe_request('get', url, DEFAULT_HEADERS, params=params, api_name="지역 목록 API - 법정동")

        if response_data and 'result' in response_data and 'list' in response_data['result']:
            legal_dong_list_raw = response_data['result']['list']
            legal_dong_names = []
            for legal_dong_data in legal_dong_list_raw:
                name = legal_dong_data.get('CortarNm')
                code = legal_dong_data.get('CortarNo')
                # 좌표 정보는 legal_dong_data 바로 아래에 포함되어 있습니다: MapXCrdn, MapYCrdn
                if name and code:
                    legal_dong_names.append(name)
                    self._regions_data['legal_dong'][name] = {
                        'code': code,
                        'lat_center': float(legal_dong_data.get('MapYCrdn')), # 위도 (MapYCrdn 사용)
                        'lon_center': float(legal_dong_data.get('MapXCrdn')), # 경도 (MapXCrdn 사용)
                    }
            self.legal_dong_combobox['values'] = legal_dong_names
            self.update_status(f"'{selected_gungu_name}'의 법정동 목록 로드 완료.")
        else:
            self.update_status(f"⛔️ '{selected_gungu_name}'의 법정동 목록 로드 실패. (응답 구조 불일치 또는 데이터 없음)")
            self.master.after(0, lambda: messagebox.showerror("오류", f"'{selected_gungu_name}' 법정동 목록을 가져오지 못했습니다. API 응답 구조를 확인해주세요."))

    # 법정동 선택 시 좌표 자동 세팅
    def on_legal_dong_selected(self, event):
        selected_legal_dong_name = self.legal_dong_combobox.get()
        legal_dong_info = self._regions_data['legal_dong'].get(selected_legal_dong_name)

        if not legal_dong_info:
            return
        
        self.cortar_no_var.set(legal_dong_info.get('code', '')) 
        
        center_lat = legal_dong_info.get('lat_center')
        center_lon = legal_dong_info.get('lon_center')

        # MapXCrdn/MapYCrdn만 제공되므로, 고정된 크기의 가상 바운딩 박스 생성
        if center_lat is not None and center_lon is not None:
            # 현재 선택된 줌 레벨을 가져옴 (입력값이 int가 아닐 수 있으므로 float로 변환 후 int)
            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14 # 기본값 설정

            # z_level에 따른 델타 값 조회. 없으면 기본값 14 델타 사용
            deltas = self._Z_LEVEL_DELTAS.get(z_level, self._Z_LEVEL_DELTAS[14]) 
            lat_delta = deltas['lat_delta']
            lon_delta = deltas['lon_delta']

            btm_val = center_lat - lat_delta
            top_val = center_lat + lat_delta
            lft_val = center_lon - lon_delta
            rgt_val = center_lon + lon_delta
            
            # StringValued에 설정할 때 float -> string 변환
            self.coord_vars['lat'].set(str(center_lat))
            self.coord_vars['lon'].set(str(center_lon))
            self.coord_vars['btm'].set(str(btm_val))
            self.coord_vars['lft'].set(str(lft_val))
            self.coord_vars['top'].set(str(top_val))
            self.coord_vars['rgt'].set(str(rgt_val))
            
            self.update_status(f"'{selected_legal_dong_name}' (코드: {legal_dong_info.get('code')}) 선택 완료. 좌표 자동 입력됨.")
        else:
            # 중심 좌표가 없는 경우 모든 좌표 필드를 비웁니다.
            for var in self.coord_vars.values():
                var.set('')
            self.update_status(f"'{selected_legal_dong_name}' 선택 완료. 중심 좌표 정보가 없어 자동 입력 불가.")
            self.master.after(0, lambda: messagebox.showwarning("좌표 누락", "선택된 법정동의 중심 좌표 정보를 찾을 수 없어 경계 좌표를 설정할 수 없습니다."))

    # 조회 시작용 스레드 시작 함수 (이전과 동일)
    def start_fetch_thread(self):
        if hasattr(self, 'fetch_thread') and self.fetch_thread.is_alive():
            self.master.after(0, lambda: messagebox.showwarning("경고", "이미 매물 조회 작업이 진행 중입니다."))
            return
        self._stop_flag = False
        self.master.after(0, lambda: self.query_button.config(state=tk.DISABLED))
        self.master.after(0, lambda: self.stop_button.config(state=tk.NORMAL))
        self.save_button.config(state=tk.DISABLED) # 조회 시작시 저장 버튼 비활성화
        self.update_status("매물 조회 시작...")
        self.fetched_article_data = [] # 조회 시작 시 데이터 초기화

        self.fetch_thread = threading.Thread(target=self.fetch_articles, daemon=True)
        self.fetch_thread.start()

    # 중지 함수 (이전과 동일)
    def stop_fetch(self):
        self._stop_flag = True
        self.update_status("중지 요청됨. 현재 페이지 작업 완료 후 중단됩니다.")
        self.master.after(0, lambda: self.stop_button.config(state=tk.DISABLED))

    # 매물 조회 함수
    def fetch_articles(self):
        cortar_no = self.cortar_no_var.get().strip() 
        max_pages = self.max_pages_var.get()

        lat = self.coord_vars['lat'].get().strip()
        lon = self.coord_vars['lon'].get().strip()
        btm = self.coord_vars['btm'].get().strip()
        lft = self.coord_vars['lft'].get().strip()
        top = self.coord_vars['top'].get().strip()
        rgt = self.coord_vars['rgt'].get().strip()

        # 🚨 부동산 유형 체크박스 상태를 기반으로 최종 rletTpCd 생성
        selected_rlet_types = []
        for name, var in self.rlet_type_vars.items():
            if var.get(): # 체크박스가 선택되었다면
                api_code_str = self._RLET_TYPE_CODES.get(name, '') 
                if api_code_str:
                    selected_rlet_types.extend(api_code_str.split(':'))
        
        rlet_tp_cd_final = ":".join(sorted(list(set(selected_rlet_types))))

        # 🚨 거래 유형 체크박스 상태를 기반으로 최종 tradTpCd 생성
        selected_trad_types = []
        for name, var in self.trad_type_vars.items():
            if var.get(): 
                api_code = self._TRAD_TYPE_CODES.get(name, '')
                if api_code:
                    selected_trad_types.append(api_code)
        
        trad_tp_cd_final = ":".join(sorted(list(set(selected_trad_types))))

        # 🚨 평형대 체크박스 상태를 기반으로 spcMin/spcMax 생성
        min_spc = float('inf')  
        max_spc = 0.0

        is_pyeong_selected = False # 평형대 선택 여부 플래그
        for name, var in self.pyeong_type_vars.items():
            if var.get():
                is_pyeong_selected = True
                pyeong_range = self._PYEONG_TYPE_RANGES[name]
                min_val = pyeong_range.get('spcMin', 0) 
                max_val = pyeong_range.get('spcMax', float('inf')) 

                min_spc = min(min_spc, min_val)
                max_spc = max(max_spc, max_val)
        
        params_spcMin = None
        params_spcMax = None
        if is_pyeong_selected:
            params_spcMin = str(int(min_spc)) if min_spc != float('inf') else None 
            params_spcMax = str(int(max_spc)) if max_spc != float('inf') else None 

        if not cortar_no:
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "법정동을 선택하세요."))
            self.reset_buttons_state()
            return
        if max_pages <= 0:
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "최대 페이지 수는 1 이상이어야 합니다."))
            self.reset_buttons_state()
            return
        
        if not rlet_tp_cd_final: 
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 부동산 유형을 선택해주세요."))
            self.reset_buttons_state()
            return

        if not trad_tp_cd_final: 
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 거래 유형을 선택해주세요."))
            self.reset_buttons_state()
            return

        if not is_pyeong_selected: 
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "하나 이상의 평형대를 선택해주세요."))
            self.reset_buttons_state()
            return
        
        if not (lat and lon and btm and lft and top and rgt): 
            self.master.after(0, lambda: messagebox.showwarning("입력 오류", "선택된 법정동의 지도 좌표 정보가 불완전합니다. (모든 좌표값 필요)"))
            self.reset_buttons_state()
            return

        self.master.after(0, lambda: [self.tree.delete(i) for i in self.tree.get_children()])
        total_count = 0
        self.fetched_article_data = [] # 조회 시작 시 데이터 초기화
        self.update_status("매물 조회 중...")

        for page in range(1, max_pages + 1):
            if self._stop_flag:
                self.update_status("사용자 요청으로 조회 중단됨.")
                break

            try:
                z_level = int(self.z_level_var.get())
            except ValueError:
                z_level = 14 # 기본값

            params = {
                "itemId": "", "mapKey": "", "lgeo": "", "showR0": "",
                "rletTpCd": rlet_tp_cd_final, 
                "tradTpCd": trad_tp_cd_final, 
                "z": str(z_level), # 줌 레벨 적용
                "lat": lat, "lon": lon, "btm": btm, "lft": lft, "top": top, "rgt": rgt,
                "totCnt": "0",
                "cortarNo": cortar_no,
                "sort": "rank",
                "page": page
            }
            if params_spcMin is not None:
                params['spcMin'] = params_spcMin
            if params_spcMax is not None:
                params['spcMax'] = params_spcMax
            
            response = self._safe_request('get', GET_API_URL, DEFAULT_HEADERS, params=params, api_name="매물 조회 API")
            if response == "RETRY":
                page -= 1
                continue
            
            if not response or 'body' not in response: 
                self.update_status("매물 조회 실패 또는 매물 없음. 응답에 body 키가 없거나 비어있습니다.")
                break

            articles = response['body']
            if not articles:
                self.update_status("더 이상 매물이 존재하지 않습니다.")
                break

            for art in articles:
                # Treeview에 표시할 데이터
                tree_values = (
                    art.get('atclNo', 'N/A'),
                    art.get('atclNm', 'N/A'),
                    art.get('tradTpNm', 'N/A'),
                    art.get('hanPrc', 'N/A'),
                    art.get('prc', 'N/A'),
                    art.get('rentPrc', 'N/A'),
                    art.get('flrInfo', 'N/A'),
                    art.get('spc1', 'N/A'),
                    art.get('spc2', 'N/A'),
                    art.get('direction', 'N/A'),
                    art.get('cortarNo', 'N/A')
                )
                self.master.after(0, lambda a=tree_values: self.tree.insert("", "end", values=a))

                # pandas 저장을 위한 원본 데이터를 저장
                self.fetched_article_data.append(art)


            total_count += len(articles)

            if page < max_pages:
                delay = random.uniform(0, 1) 
                self.update_status(f"{page}페이지 완료. 다음 페이지 조회 전 {delay:.2f}초 대기 중...")
                time.sleep(delay)

        self.update_status(f"총 {total_count}개의 매물을 테이블에 표시했습니다.")
        self.reset_buttons_state()
        self.master.after(0, lambda: messagebox.showinfo("완료", f"총 {total_count}개의 매물을 가져왔습니다."))
        if self.fetched_article_data: # 데이터가 있을 경우에만 저장 버튼 활성화
            self.master.after(0, lambda: self.save_button.config(state=tk.NORMAL))


    def reset_buttons_state(self):
        self.master.after(0, lambda: self.query_button.config(state=tk.NORMAL))
        self.master.after(0, lambda: self.stop_button.config(state=tk.DISABLED))

    # 파일 저장 경로 선택 다이얼로그
    def select_save_path(self):
        selected_dong_name = self.legal_dong_combobox.get()
        if not selected_dong_name:
            selected_dong_name = "매물"
        
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        default_filename = f"{selected_dong_name}_{timestamp}.csv"

        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
            initialfile=default_filename # 기본 파일명 제안
        )
        if file_path:
            self.save_path_var.set(file_path)

    # 조회된 데이터를 CSV 파일로 저장
    def save_articles_to_file(self):
        if not self.fetched_article_data:
            messagebox.showwarning("저장 오류", "저장할 매물 데이터가 없습니다.")
            return
        
        file_path = self.save_path_var.get()
        if not file_path:
            messagebox.showwarning("저장 오류", "저장 경로를 입력해주세요.")
            return

        try:
            df = pd.DataFrame(self.fetched_article_data)
            
            # Treeview 컬럼 순서 및 한글명으로 맞추기 (선택 사항)
            # 매핑 정의
            column_mapping = {
                'atclNo': "매물번호", 'atclNm': "매물명", 'tradTpNm': "거래종류", 
                'hanPrc': "한글가격", 'prc': "매매/전세가", 'rentPrc': "월세가", 
                'flrInfo': "층정보", 'spc1': "공급(㎡)", 'spc2': "전용(㎡)", 
                'direction': "방향", 'cortarNo': "법정동코드"
            }
            # DataFrame에서 필요한 컬럼만 추출하고 순서 정렬
            cols_to_extract = list(column_mapping.keys())
            df_filtered = df[cols_to_extract].copy()
            # 컬럼명 변경
            df_filtered = df_filtered.rename(columns=column_mapping)
            
            # CSV로 저장 (UTF-8 인코딩)
            df_filtered.to_csv(file_path, index=False, encoding='utf-8-sig') 
            messagebox.showinfo("저장 완료", f"매물 데이터가 '{file_path}'에 성공적으로 저장되었습니다.")
            self.update_status(f"데이터를 '{file_path}'에 저장했습니다.")
        except Exception as e:
            messagebox.showerror("저장 오류", f"파일 저장 중 오류가 발생했습니다: {e}")
            self.update_status(f"⛔️ 저장 오류: {e}")


if __name__ == "__main__":
    current_date = datetime.date.today() # 오늘 날짜를 가져옵니다.

    # 1. 프로그램 만료일 확인 (기존 로직)
    if current_date > EXPIRATION_DATE:
        messagebox.showerror("프로그램 만료", f"프로그램 사용 기한이 {EXPIRATION_DATE}로 만료되었습니다.\n더 이상 프로그램을 사용할 수 없습니다.")
        import sys
        sys.exit() # 프로그램 강제 종료
    
    # 2. 만료 예정일 알림 확인 (새로운 로직 추가)
    # 만료일 한 달 전 (30일 전) 기준일을 계산
    # datetime.timedelta는 날짜 간의 간격을 표현합니다.
    expiration_warning_date = EXPIRATION_DATE - datetime.timedelta(days=30) 

    if current_date >= expiration_warning_date and current_date <= EXPIRATION_DATE: # 만료일 포함하여 알림
        remaining_days = (EXPIRATION_DATE - current_date).days # 남은 일수 계산
        messagebox.showwarning(
            "사용 기한 임박", 
            f"⚠️ 프로그램 사용 기한이 {EXPIRATION_DATE}까지입니다. (남은 기간: {remaining_days}일)\n"
            "미리 준비해 주세요."
        )

    # 만료되지 않았다면, 정상적으로 Tkinter 애플리케이션을 시작합니다.
    root = tk.Tk()
    app = NaverRealEstateApp(root)
    root.mainloop()


--- API 요청 시작: 지역 목록 API - 시도 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '0000000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 시군구 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '1100000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 시군구 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4400000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 법정동 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4425000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 법정동 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4476000000', 'mycortarNo': ''}
  Status Code: 200

--- API 요청 시작: 지역 목록 API - 법정동 ---
  URL: https://m.land.naver.com/map/getRegionList
  Method: GET
  Params: {'cortarNo': '4481000